# Read ICON HAM-lite data in ICON grid using uxarray

`uxarray` is a python library that extends `xarray` to working with "unstructured" grid systems, including the icosohedral grid used by ICON.

In this notebook we explore reading and plotting ICON data using uxarray

In [1]:
import pathlib

import xarray as xr
import uxarray

## Grid file

First, we need to load the grid that the data is stored on. This is stored in the invariants directory

In [2]:
grid_file = pathlib.Path("/work/scratch-pw2/wkjones/hamlite/invariants/icon_grid_G.nc")

In [3]:
grid_r2b9 = uxarray.open_grid(grid_file, chunks={})

In [4]:
grid_r2b9

<uxarray.Grid>
Original Grid Type: ICON
Grid Dimensions:
  * n_node: 10485762
  * n_edge: 31457280
  * n_face: 20971520
  * n_max_face_nodes: 3
  * n_max_face_edges: 3
  * n_max_face_faces: 3
  * two: 2
Grid Coordinates (Spherical):
  * node_lon: (10485762,)
  * node_lat: (10485762,)
  * edge_lon: (31457280,)
  * edge_lat: (31457280,)
  * face_lon: (20971520,)
  * face_lat: (20971520,)
Grid Coordinates (Cartesian):
Grid Connectivity Variables:
  * edge_face_connectivity: (31457280, 2)
  * face_face_connectivity: (20971520, 3)
  * face_edge_connectivity: (20971520, 3)
  * edge_node_connectivity: (31457280, 2)
  * face_node_connectivity: (20971520, 3)
Grid Descriptor Variables:

## Load data into xarray

Next, we can load the data and turn it into a UXarray dataarray

In [5]:
burden_file = pathlib.Path("/work/scratch-pw2/wkjones/hamlite/run_20200229T000000-20200304T235920/hamlite_bdn_ml_20200229T000000Z.nc")

Simply loading with xarray provides us with no geospatial information:

In [6]:
burdens_ds = xr.open_dataset(burden_file)
burdens_ds

<xarray.Dataset> Size: 33GB
Dimensions:         (time: 95, ncells: 20971520, vertices: 3)
Coordinates:
  * time            (time) datetime64[ns] 760B 2020-02-29T00:15:00 ... 2020-0...
    clon            (ncells) float64 168MB ...
    clat            (ncells) float64 168MB ...
Dimensions without coordinates: ncells, vertices
Data variables:
    clon_bnds       (ncells, vertices) float64 503MB ...
    clat_bnds       (ncells, vertices) float64 503MB ...
    burden_num_qdu  (time, ncells) float32 8GB ...
    burden_num_qss  (time, ncells) float32 8GB ...
    burden_num_qca  (time, ncells) float32 8GB ...
    burden_num_qsu  (time, ncells) float32 8GB ...
Attributes:
    CDI:                  Climate Data Interface version 2.2.0 (https://mpime...
    Conventions:          CF-1.6
    number_of_grid_used:  15
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    title:                ICON simulation
    source:               git@gitlab.dkrz.de:icon/icon-mpim.git@4ce6927f298ef...
    history:              ./icon at 20240924 012336
    references:           see MPIM/DWD publications
    comment:              Philipp Weiss (b381642) on l40646 (Linux 4.18.0-477...

But we can convert this to a uxarray object to make it geospatially aware:

In [7]:
burdens_uxds = uxarray.UxDataset.from_xarray(burdens_ds, uxgrid=grid_r2b9)
burdens_uxds

<xarray.UxDataset> Size: 33GB
Dimensions:         (time: 95, n_face: 20971520, vertices: 3)
Coordinates:
  * time            (time) datetime64[ns] 760B 2020-02-29T00:15:00 ... 2020-0...
    clon            (n_face) float64 168MB ...
    clat            (n_face) float64 168MB ...
Dimensions without coordinates: n_face, vertices
Data variables:
    clon_bnds       (n_face, vertices) float64 503MB ...
    clat_bnds       (n_face, vertices) float64 503MB ...
    burden_num_qdu  (time, n_face) float32 8GB ...
    burden_num_qss  (time, n_face) float32 8GB ...
    burden_num_qca  (time, n_face) float32 8GB ...
    burden_num_qsu  (time, n_face) float32 8GB ...

## Plotting

Now lets plot the sea-salt burden at one time step:

In [8]:
burdens_uxds.burden_num_qss[0].plot()

:Image   [x,y]   (x_y burden_num_qss)